In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as wg
from IPython.display import display
from apm import *

In [2]:
# define server and application name
s = 'http://byu.apmonitor.com'
a = 'mpc_widget'

# initialize MPC
def mpc_init(s,a):
    # clear prior and load new application
    apm(s,a,'clear all')
    apm_load(s,a,'mpc.apm')
    csv_load(s,a,'mpc.csv')

    # set global options
    apm_option(s,a,'apm.imode',6)
    apm_option(s,a,'apm.mv_type',1)
    apm_option(s,a,'apm.cv_type',1)
    apm_option(s,a,'apm.mv_step_hor',1)
    apm_option(s,a,'apm.time_shift',0)
    apm_option(s,a,'apm.solver',1)
    apm_option(s,a,'apm.max_iter',100)

    # declare MV and CV
    apm_info(s,a,'MV','u')
    apm_info(s,a,'CV','y')

    # default options for MV
    apm_option(s,a,'u.status',1)
    apm_option(s,a,'u.cost',-0.1)
    apm_option(s,a,'u.dcost',0.1)
    apm_option(s,a,'u.dmax',2.0)

    # default options for CV
    apm_option(s,a,'y.status',1)
    apm_option(s,a,'y.cost',0)
    apm_option(s,a,'y.sphi',10.0)
    apm_option(s,a,'y.splo',8.0)
    apm_option(s,a,'y.tau',5.0)
    apm_option(s,a,'y.tr_init',2)
    apm_option(s,a,'y.tr_open',3)
    apm_option(s,a,'y.wsphi',10.0)
    apm_option(s,a,'y.wsplo',10.0)
    
    output = apm(s,a,'solve')
    return output

print(mpc_init(s,a))

apm 202.80.219.198_mpc_widget <br><pre> ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            5
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Variable time shift OFF
 Number of state variables:            266
 Number of total equations: -          252
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :             14
 
 ----------------------------------------------
 Dynamic Control with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  9.34170E+01  4.00000E+00
    1 -2.25227E+00  1.09091E-10
    2 -2.25227E+0

### Manipulated Variable Tuning

* COST = (+) minimize MV, (-) maximize MV 
* DCOST = penalty for MV movement 
* DMAX = maximum that MV can move each cycle 
* MV_STEP_HOR = default 1, higher integers allow MV to move less frequently

In [3]:
def mpc_mv_tuning(cost,dcost,dmax,mv_step):
    apm_option(s,a,'u.cost',cost)
    apm_option(s,a,'u.dcost',dcost)
    apm_option(s,a,'u.dmax',dmax)
    apm_option(s,a,'u.mv_step_hor',mv_step)
    
    # solve mpc
    output = apm(s,a,'solve')
    
    # retrieve solution
    z = apm_sol(s,a)
    
    plt.figure(1,figsize=(10,7))
    plt.subplot(2,1,1)
    plt.plot(z['time'],z['u'],'b-',label=r'$u$')
    plt.legend(loc='best')
    plt.ylabel('MV')

    plt.subplot(2,1,2)
    plt.plot(z['time'],z['y.tr_hi'],'r:',linewidth=3,label=r'$y_{hi}$')
    plt.plot(z['time'],z['y'],'k',linewidth=3,label=r'$y$')
    plt.plot(z['time'],z['y.tr_lo'],'g:',linewidth=3,label=r'$y_{lo}$')
    plt.legend(loc='best')
    plt.ylabel('CV')
    plt.xlabel('Time (sec)')
    
cost_slide = wg.FloatSlider(value=0.0,min=-1.0,max=1.0,step=0.1)
dcost_slide = wg.FloatSlider(value=0.1,min=0.0,max=1.0,step=0.1)
dmax_slide = wg.FloatSlider(value=1.0,min=0.0,max=1.0,step=0.1)
mv_step_slide = wg.FloatSlider(value=1,min=1,max=10,step=1)
wg.interact(mpc_mv_tuning, cost=cost_slide, dcost=dcost_slide, dmax=dmax_slide, mv_step=mv_step_slide)

interactive(children=(FloatSlider(value=0.0, description='cost', max=1.0, min=-1.0), FloatSlider(value=0.1, de…

<function __main__.mpc_mv_tuning(cost, dcost, dmax, mv_step)>

### Controlled Variable Tuning

* COST = (+) minimize CV, (-) maximize CV
* SP = setpoint for the controller, translated to SPLO / SPHI for l1-norm dead-band
* TAU = time constant of reference trajectory
* TR_INIT = 0 for constant dead-band, 1 or 2 for reference trajectory, 2 re-initializes initial condition every cycle
* TR_OPEN = relative opening of the initial trajectory relative to the SPHI - SPLO end difference

In [4]:
def mpc_cv_tuning(cost,sp,tau,tr_init,tr_open):
    gap = 2.0
    apm_option(s,a,'y.cost',cost)
    apm_option(s,a,'y.sphi',sp+gap/2)
    apm_option(s,a,'y.splo',sp-gap/2)
    apm_option(s,a,'y.tau',tau)
    apm_option(s,a,'y.tr_init',tr_init)
    apm_option(s,a,'y.tr_open',tr_open)
    
    # solve mpc
    output = apm(s,a,'solve')
    
    # retrieve solution
    z = apm_sol(s,a)
    
    plt.figure(1,figsize=(10,7))
    plt.subplot(2,1,1)
    plt.plot(z['time'],z['u'],'b-',label=r'$u$')
    plt.legend(loc='best')
    plt.ylabel('MV')

    plt.subplot(2,1,2)
    plt.plot(z['time'],z['y.tr_hi'],'r:',linewidth=3,label=r'$y_{hi}$')
    plt.plot(z['time'],z['y'],'k',linewidth=3,label=r'$y$')
    plt.plot(z['time'],z['y.tr_lo'],'g:',linewidth=3,label=r'$y_{lo}$')
    plt.legend(loc='best')
    plt.ylabel('CV')
    plt.xlabel('Time (sec)')
    
cost_slide = wg.FloatSlider(value=0.0,min=-1.0,max=1.0,step=0.1)
sp_slide = wg.FloatSlider(value=9.0,min=0.0,max=20.0,step=1.0)
tau_slide = wg.FloatSlider(value=5.0,min=1.0,max=10.0,step=1.0)
tr_init_slide = wg.FloatSlider(value=2.0,min=0.0,max=2.0,step=1.0)
tr_open_slide = wg.FloatSlider(value=3.0,min=0.0,max=10.0,step=1.0)
wg.interact(mpc_cv_tuning, cost=cost_slide, sp=sp_slide, tau=tau_slide, tr_init=tr_init_slide, tr_open=tr_open_slide)

interactive(children=(FloatSlider(value=0.0, description='cost', max=1.0, min=-1.0), FloatSlider(value=9.0, de…

<function __main__.mpc_cv_tuning(cost, sp, tau, tr_init, tr_open)>